In [98]:
from pygame_2d import*
import numpy as np
import const

screen = np.zeros((720, 1280, 3), dtype = np.uint8)



In [141]:

high, low = 1,0

alpha = 0, 2*math.pi    # góc quay 
rVelo = PLAYER_SETTING.MIN_ROTATION_VELO, PLAYER_SETTING.MAX_ROTATION_VELO  # vận tốc quay 
fwVelo = -PLAYER_SETTING.MAX_FORWARD_VELO, PLAYER_SETTING.MAX_FORWARD_VELO    # vận tốc tiến



In [131]:

gamma = 0.99
alpha = 0.1
epsilon = 1
epsilon_decay = epsilon / 4000
n_epsilondes = 10000



In [ ]:
N_LIDARSIGNAL = 4   # 0,1,2 and 3 is inf
N_SAPCELIDAR = 3    # 0,1,2
random.seed(13)
lidars = np.random.randint(0, 360, size= 360)

bin_lidarsignal = np.linspace(0, 360, num= N_LIDARSIGNAL, endpoint= True)
bin_lidarsignal = np.delete(bin_lidarsignal, 0)
lidars = np.digitize(lidars, bin_lidarsignal)

lidars[lidars == 3] = INFINITY
const_lidars = lidars
temp_lidars = lidars


In [130]:
bin_lidarspace = np.array([45, 135, 180])
lidars = np.array_split(temp_lidars, bin_lidarspace)

# dis_lidars = {"0": (np.amin(lidars[0]), lidars[0]),
#               "1": (np.amin(lidars[1]), lidars[1]),
#               "2": (np.amin(lidars[2]), lidars[2]),
#               "3": INFINITY}

section_lidars_min = [np.amin(lidars[0]), np.amin(lidars[1]), np.amin(lidars[2]), INFINITY]
lidarStateVector = np.array(section_lidars_min)
lidarStateVector

array([    0,     0,     0, 99999], dtype=int64)

In [147]:


# set các khoảng rời rạc
ALPHA_SPACE = 20
FWVELO_SPACE = 4
RVELO_SPACE = 4
high, low = 1, 0
alpha_space = 0, 2*math.pi
fwVelo_space = -PLAYER_SETTING.MAX_FORWARD_VELO, PLAYER_SETTING.MAX_FORWARD_VELO
rVelo_space = PLAYER_SETTING.MIN_ROTATION_VELO, PLAYER_SETTING.MAX_ROTATION_VELO

lowState = np.array(
    [alpha_space[0], fwVelo_space[0], rVelo_space[0]], dtype=float)
upState = np.array(
    [alpha_space[1], fwVelo_space[1], rVelo_space[1]], dtype=float)

infoState_shape = (ALPHA_SPACE, FWVELO_SPACE, RVELO_SPACE)

bins = []
for i in range(3):
    item = np.linspace(
        lowState[i],
        upState[i],
        num=infoState_shape[i],
        endpoint=False)
    item = np.delete(item, 0)
    bins.append(item)
    print(bins[i])

def get_discrete_state(s):
    new_s = []
    for i in range(3):
        new_s.append(np.digitize(s[i], bins[i]))
    return new_s

infoStateVector = get_discrete_state([0.3895725, 99, -0.658345])
infoStateVector


[0.31415927 0.62831853 0.9424778  1.25663706 1.57079633 1.88495559
 2.19911486 2.51327412 2.82743339 3.14159265 3.45575192 3.76991118
 4.08407045 4.39822972 4.71238898 5.02654825 5.34070751 5.65486678
 5.96902604]
[-50.   0.  50.]
[-0.5  0.   0.5]


[1, 3, 0]

In [151]:
new_observation = np.concatenate((infoStateVector, lidarStateVector))
new_observation

array([    1,     3,     0,     0,     0,     0, 99999], dtype=int64)

In [152]:
q_table = np.zeros(new_observation.shape + (ACTION_SPACE, ))
print(q_table.shape)

(7, 6)


In [ ]:

def pick_sample(state):
    if np.random.random() > epsilon:
        action = np.argmax(q_table[tuple(state)])
    else:
        action = np.random.randint(0, ACTION_SPACE)
    return action

def step(action):
    pass

reward_records = []
for i in range(n_epsilondes):
    done = False
    total_reward = 0
    
    game = PyGame2D(screen=screen)
    state = game.observe()
    
    s_dis = get_discrete_state(state)
    while not done:
        action = pick_sample(s_dis, i)
        #! ... (thêm) sau khi thực hiện action thì cập nhật lại...
        #! sửa lại hàm action bên PyGame2D 
        state, reward, done = game.observe(), game.evaluate(), game.is_done()
        
        s_dis_next = get_discrete_state(state)

        # Update Q-Table
        maxQ = np.max(q_table[tuple(s_dis_next)])
        q_table[tuple(s_dis)][action] += alpha * (reward + gamma * maxQ - q_table[tuple(s_dis)][action])

        s_dis = s_dis_next
        total_reward += reward

    # Update epsilon for each episode
    if epsilon - epsilon_decay >= 0:
        epsilon -= epsilon_decay

    # Record total rewards in episode 
    print("Run episode {} with rewards {}".format(i, total_reward), end="\r")
    reward_records.append(total_reward)

print("\nDone")
